# COVID-19 Doctor Chatbot - GPT2



In [ ]:
!pip -q install transformers
!pip install gdown
!pip install "nltk==3.4.5"

     |████████████████████████████████| 1.4MB 12.3MB/s 
     |████████████████████████████████| 890kB 50.0MB/s 
     |████████████████████████████████| 2.9MB 50.0MB/s 


## Set up drive

In [117]:
!gdown https://drive.google.com/uc?id=1-Cmprk6ZOvSrOCxoZldu-qCXuPGyFjtz

Downloading...
From: https://drive.google.com/uc?id=1-Cmprk6ZOvSrOCxoZldu-qCXuPGyFjtz
To: /content/drive/MyDrive/Colab/GPT2-finetune/pytorch_model.bin
510MB [00:02, 178MB/s]


In [118]:
!mv pytorch_model.bin output-small-save/

## Dataset setup

Set up dataset and view first 5 items in training and validation sets

In [122]:
import dataset

df_trn, df_val = dataset.make_dataset(path_to_train_data="data/train_data.json", path_to_validation_data="data/validate_data.json")

> /content/drive/My Drive/Colab/GPT2-finetune/dataset.py(12)make_dataset()
-> f_validate.close()
(Pdb) c


In [123]:
df_trn.head(5)

,response,context
0,"Hello, I understand your concern. I just have ...","Hello doctor, I get a cough for the last few d..."
1,"Hello, I can understand your concern.In my opi...","Hello doctor, I am suffering from coughing, th..."
2,Hello. Anxiety can manifest itself in physical...,"Hello doctor,I am a 23-year-old man. I have an..."
3,"Hello,please answer the following:Any travel h...","Hello doctor,Last night I was getting chills, ..."
4,Hello and welcome to Ask A Doctor service.I ha...,"Hi, I am Chaitanya, 27 years old. I use to swi..."


In [124]:
df_val.head(5)

,response,context
0,Corona-virus. At 33 you may not need testing. ...,I have a constant cough and my chest has now b...
1,Less likely. Recommended to stay 6 feet apart....,If someone has carona virus and iam passing by...
2,"Test Please stay at home, rest, drink fluids...",I am concerned that I’m showing symptoms of co...
3,Death. At your age the risk of death is the fo...,What are my chances of becoming seriously ill ...
4,Unknown but low Based on current data it is ...,Nervous about coronavirus. I am 26 years old a...


## Evaluating


In [72]:
!python main.py

2020-12-04 22:00:10.930707: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/04/2020 22:00:12 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:852: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
12/04/2020 22:00:23 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x7f959d6c1160>
12/04/2020 22:00:23 - INFO - __main__ -   Evaluate the following checkpoints: ['output-small-save']
12/04/2020 22:00:29 - INFO - utils -   Creating features from dataset file at cached
12/04/2020 22:00:29 - INFO - utils -   Sav

### Show up to 5 test results

In [79]:
test_chatbot = []

for i in range(5): #len(test_query)
  tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
  model = AutoModelWithLMHead.from_pretrained('output-small-save')
  # append the new user input tokens to the chat history
  bot_input_ids = tokenizer.encode(test_query[i] + tokenizer.eos_token, return_tensors='pt')
  print("User: {} \n".format(test_query[i]))

  # generated a response while limiting the total chat history to 1000 tokens, 
  chat_history_ids = model.generate(
      bot_input_ids, max_length=100,
      pad_token_id=tokenizer.eos_token_id,  
      no_repeat_ngram_size=3,       
      do_sample=True, 
      top_k=10, 
      top_p=0.7,
      temperature = 0.8
  )

  # pretty print last ouput tokens from bot
  print("Chatbot: {} \n\n".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
  test_chatbot.append(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True))

print(len(test_chatbot))


/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:852: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


User: I have all the symptoms except fever, I went to Medicross and Dr said I can get tested if I want to I'm not sure if I should. She gave me antibiotics Klacid XL 500mg, she said I can take it if I feel worse I'm worried it will make immune system bad? 

Chatbot: Take a test.    You will likely need to take the test. 


User: I have pain/discomfort in my lungs. I don't experience simultaneous on both lungs and it not always at the hame position. I don't have a head nor do I have high temperature. I sneeze and cough maybe once a day. Do I have corona, should I get tested? 

Chatbot: I can understand your concern. I have no idea what you are experiencing. If you have a cold or other respiratory disease, consult your local physician. 


User: I travelled to Mauritius and do not have symptoms. Should I get tested for covid19? 

Chatbot: Yes, if you are in contact with someone with Covid19. You should be tested for Covid 19.    If you have a fever and or fever, get tested and get tested.

## Generate test results


In [95]:
with open('gpt2-results.txt') as f:
    test_chatbot = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
test_chatbot = [x.strip() for x in content] 

## Metrics

In [83]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk.translate.nist_score import sentence_nist

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [105]:
import metrics
bleu_2, bleu_4, meteor, nist_2, nist_4 = metrics.get_metrics(test_chatbot, test_response)

In [106]:
 bleu_2, bleu_4, meteor, nist_2, nist_4

(0.1987791535170549,
 0.10355670162282384,
 0.22840520757502006,
 0.9900288417339577,
 1.0254752183668514)

## Interactive Chat

A variety of methods can be used in responces generation. You can find more details about these methods by this [link](https://huggingface.co/blog/how-to-generate). 

In [109]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelWithLMHead.from_pretrained('output-small-save')

# Let's chat for 1 line
for step in range(1):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=100,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=10, 
        top_p=0.7,
        temperature = 0.8
    )
    
    # pretty print last ouput tokens from bot
    print("Chatbot: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:852: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


>> User:Hi I am 39 years old and returned from Germany 19 days ago. Yesterday I started getting a sore throat, runny nose. Today I have sinus pressure and a headache with a blocked nose, throat seems to be improving. Should I get tested. If so how?
Chatbot: Hello,    If you have a sore nose, you should definitely get tested for COVID-19.   Would you like to video or text chat with me?
